# 单链路计算图自动反向传播

> 原理：简而言之，如果只给函数类实例反向传播的方法，那么就需要手动一个一个调用函数的反向传播方法，如果用变量调用反向传播方法，并给之以在调用变量类实例的反向传播方法时，可以自动调用产生他函数的方法

In [2]:
#为变量类Variable和函数类Function拓展功能

import numpy as np
class Variable:
    def __init__(self,data):
        self.data = data        
        self.grad = None
        #新拓展
        self.creator = None         #记录变量的创造函数

        #新拓展
    def set_creator(self,func):
        self.creator = func

        #新拓展
    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()


class Function:
    def __call__(self,input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)    #设置产生变量的creator属性，不可省略，与Variable类反向传播的第17行有强耦合性
        self.input = input
        #新拓展
        self.output = output       #思考可以省略吗?
        return output
    
    def forward(self,input):
        raise NotImplementedError()
    
    def backward(self,gy):
        raise NotImplementedError()

class Square(Function):
    def forward(self,x):
        y = x**2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = x*2*gy
        return gx

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self,gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

In [7]:
#使用
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
#a = A(x)
a = Square()(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
